# Möbius‑Ladder Explorer (v3) — *Shifted Breath Cycle*

This version inserts **tiny** *after* the φ-scale reciprocal, breaking the exact Möbius cancellation:

```
x ← seed + tiny
repeat N cycles:
    x ← φ_scale·φ / x          # fission
    x ← x − tiny               # inject α here
    x ← 1/x / 9
    x ← 1/x
    x ← x / 9
```

Parameters swept:

* `seed_bits` ∈ {32, 64, 128}
* `k_tiny`   = 4   (tiny = α⁻¹ / 10ᵏ)
* `phi_scale` ∈ 0.02 … 0.20 (step 0.02)
* cycles 1 … 10

Outputs rows with relative error to α; prints any **< 1 %** hits.

In [1]:
import mpmath as mp, itertools, pandas as pd
mp.mp.dps = 120
ALPHA_INV = mp.mpf('137.035999084')
ALPHA = 1/ALPHA_INV
PHI = (1 + mp.sqrt(5)) / 2

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]
def fib_fraction(bits):
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(fib_word_bits(bits)))


In [2]:
def breath_shift(x, tiny, phi_scale):
    x = phi_scale*PHI / x   # fission
    x = x - tiny            # inject α bias
    x = 1/x / 9
    x = 1/x
    x = x / 9
    return x

## Parameter sweep with shifted breath

In [3]:
rows=[]
for bits in (32,64,128):
    seed = fib_fraction(bits)
    tiny = ALPHA_INV/1e4
    for phi_scale in [i*0.02 for i in range(1,11)]:   # 0.02 … 0.20
        x = seed + tiny
        for cycles in range(1,11):
            x = breath_shift(x,tiny,phi_scale)
            rel = abs((x-ALPHA)/ALPHA)
            rows.append(dict(bits=bits, phi_scale=phi_scale, cycles=cycles, x=mp.nstr(x,25), rel_err=float(rel)))
            if rel < 0.01:
                print('Hit <1%  | bits', bits, 'phi_scale', phi_scale, 'cycles', cycles, 'rel', rel)
df = pd.DataFrame(rows)
print('Top 15 smallest errors:')
display(df.sort_values('rel_err').head(15))


Top 15 smallest errors:


,bits,phi_scale,cycles,x,rel_err
208,128,0.02,9,0.04865524359563424795855653,5.667520
108,64,0.02,9,0.04865524359563424797294413,5.667520
8,32,0.02,9,0.04865524362996150124462248,5.667520
206,128,0.02,7,0.06075451375697252174665115,7.325555
106,64,0.02,7,0.06075451375697252176030796,7.325555
6,32,0.02,7,0.06075451378955619550609584,7.325555
204,128,0.02,5,0.07218218275175663162772924,8.891558
104,64,0.02,5,0.07218218275175663164056393,8.891558
4,32,0.02,5,0.07218218278237880501687421,8.891558
202,128,0.02,3,0.08287177097300318301341836,10.356416
